Split bounding boxes into new images

In [4]:
import cv2
import json
import os
import numpy as np

# Load annotations
with open("/Users/rfalcao/Documents/FYP/Cusp Images_081224/boxes_2.json") as f:
    data = json.load(f)

image_dir = "/Users/rfalcao/Documents/FYP/Cusp Images_081224/Clear"
output_dir = "/Users/rfalcao/Documents/FYP/Cusp Images_081224/Boxed"

os.makedirs(output_dir, exist_ok=True)

for img_info in data["images"]:
    img_path = os.path.join(image_dir, img_info["file_name"])
    img = cv2.imread(img_path)

    if img is None:
        print(f"Could not read image: {img_info['file_name']}")
        continue

    for ann in data["annotations"]:
        if ann["image_id"] != img_info["id"]:
            continue

        x, y, w, h = ann["bbox"]
        angle = ann["attributes"].get("rotation", 0)

        center = (x + w / 2, y + h / 2)

        # Get rotation matrix
        M = cv2.getRotationMatrix2D(center, angle, 1.0)

        # Warp the image (rotate around the bbox center)
        rotated = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]), flags=cv2.INTER_CUBIC)

        # Now crop the box from the rotated image
        x1 = int(center[0] - w / 2)
        y1 = int(center[1] - h / 2)
        x2 = int(center[0] + w / 2)
        y2 = int(center[1] + h / 2)

        # Clamp to image boundaries
        x1 = max(0, x1)
        y1 = max(0, y1)
        x2 = min(rotated.shape[1], x2)
        y2 = min(rotated.shape[0], y2)

        if x2 > x1 and y2 > y1:
            cropped = rotated[y1:y2, x1:x2]
            save_path = os.path.join(output_dir, f"cusp_{ann['id']}.png")
            cv2.imwrite(save_path, cropped)
        else:
            print(f"Invalid rotated crop for annotation {ann['id']} in {img_info['file_name']}")


Separate Images into new categories, assuming you have non cusp examples

In [5]:
import os
import json
import shutil

# Paths
image_dir = "/Users/rfalcao/Documents/FYP/Cusp Images_081224/Boxed"  # Folder containing all images
annotation_file = "/Users/rfalcao/Documents/FYP/Cusp Images_081224/annotations 4/instances_default.json"  # Exported JSON file
output_dir = "/Users/rfalcao/Documents/FYP/Cusp Images_081224/annotations 4"

# Create output folders
os.makedirs(f"{output_dir}/cusp_images", exist_ok=True)
os.makedirs(f"{output_dir}/non_cusp_images", exist_ok=True)
os.makedirs(f"{output_dir}/unknown_images", exist_ok=True)

# Load annotation file
with open(annotation_file) as f:
    data = json.load(f)

# Get images with annotations
annotated_images = {}
for ann in data["annotations"]:
    img_id = ann["image_id"]
    if img_id not in annotated_images:
        annotated_images[img_id] = []
    annotated_images[img_id].append(ann["segmentation"])

# Get image file names
image_info = {img["id"]: img["file_name"] for img in data["images"]}

# Process each image
for img_id, filename in image_info.items():
    src_path = os.path.join(image_dir, filename)
    if os.path.exists(src_path):
        if img_id in annotated_images and annotated_images[img_id]:  # Has annotations
            dest_folder = "cusp_images"
        else:  # No annotations
            dest_folder = "non_cusp_images"
        shutil.move(src_path, os.path.join(output_dir, dest_folder, filename))
    else:
        print(f"Warning: {filename} is missing from {image_dir}")

# Move any leftover images (not in JSON)
for filename in os.listdir(image_dir):
    src_path = os.path.join(image_dir, filename)
    dest_path = os.path.join(output_dir, "unknown_images", filename)
    shutil.move(src_path, dest_path)

print("Images sorted successfully!")


Images sorted successfully!


In [8]:
from pycocotools.coco import COCO
import numpy as np
import cv2
import os

# Paths
annotation_file = "/Users/rfalcao/Documents/FYP/Cusp Images_081224/annotations 4/instances_default.json"
image_dir = "/Users/rfalcao/Documents/FYP/Cusp Images_081224/annotations 4/cusp_images"
mask_output_dir = "/Users/rfalcao/Documents/FYP/Cusp Images_081224/annotations 4/masks"
untagged_image_dir = "/Users/rfalcao/Documents/FYPCusp Images_081224/annotations 4/non_cusp_images"

os.makedirs(mask_output_dir, exist_ok=True)

# Load COCO JSON
coco = COCO(annotation_file)

# Process images
for img_id in coco.getImgIds():
    img_info = coco.loadImgs(img_id)[0]
    img_filename = img_info["file_name"]

    # Ensure proper mask filename (change .jpg to .png)
    mask_filename = img_filename
    mask_filepath = os.path.join(mask_output_dir, mask_filename)

    # Try loading from cusp images folder
    img_path = os.path.join(image_dir, img_filename)
    if not os.path.exists(img_path):
        img_path = os.path.join(untagged_image_dir, img_filename)

    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

    # Skip missing images
    if img is None:
        print(f"Skipping {img_filename} (not found in any folder)")
        continue

    # Create an empty mask
    mask = np.zeros(img.shape, dtype=np.uint8)

    # Get annotations (if any)
    ann_ids = coco.getAnnIds(imgIds=img_id)
    if ann_ids:
        anns = coco.loadAnns(ann_ids)
        for ann in anns:
            segmentation = ann["segmentation"]
            for seg in segmentation:
                points = np.array(seg, dtype=np.int32).reshape(-1, 2)
                cv2.fillPoly(mask, [points], 255)  # Fill the polygon mask

    # Save mask as PNG (lossless format)
    cv2.imwrite(mask_filepath, mask)

print("✅ Masks extracted and saved as PNGs!")



loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Skipping cusp_12.png (not found in any folder)
Skipping cusp_13.png (not found in any folder)
Skipping cusp_14.png (not found in any folder)
Skipping cusp_15.png (not found in any folder)
Skipping cusp_16.png (not found in any folder)
Skipping cusp_17.png (not found in any folder)
Skipping cusp_18.png (not found in any folder)
Skipping cusp_19.png (not found in any folder)


[ WARN:0@61758.202] global loadsave.cpp:268 findDecoder imread_('/Users/rfalcao/Documents/FYPCusp Images_081224/annotations 4/non_cusp_images/cusp_12.png'): can't open/read file: check file path/integrity
[ WARN:0@61758.203] global loadsave.cpp:268 findDecoder imread_('/Users/rfalcao/Documents/FYPCusp Images_081224/annotations 4/non_cusp_images/cusp_13.png'): can't open/read file: check file path/integrity
[ WARN:0@61758.203] global loadsave.cpp:268 findDecoder imread_('/Users/rfalcao/Documents/FYPCusp Images_081224/annotations 4/non_cusp_images/cusp_14.png'): can't open/read file: check file path/integrity
[ WARN:0@61758.204] global loadsave.cpp:268 findDecoder imread_('/Users/rfalcao/Documents/FYPCusp Images_081224/annotations 4/non_cusp_images/cusp_15.png'): can't open/read file: check file path/integrity
[ WARN:0@61758.204] global loadsave.cpp:268 findDecoder imread_('/Users/rfalcao/Documents/FYPCusp Images_081224/annotations 4/non_cusp_images/cusp_16.png'): can't open/read file: c

Skipping cusp_20.png (not found in any folder)
Skipping cusp_21.png (not found in any folder)


[ WARN:0@61758.872] global loadsave.cpp:268 findDecoder imread_('/Users/rfalcao/Documents/FYPCusp Images_081224/annotations 4/non_cusp_images/cusp_20.png'): can't open/read file: check file path/integrity
[ WARN:0@61758.874] global loadsave.cpp:268 findDecoder imread_('/Users/rfalcao/Documents/FYPCusp Images_081224/annotations 4/non_cusp_images/cusp_21.png'): can't open/read file: check file path/integrity


Skipping cusp_24.png (not found in any folder)
Skipping cusp_25.png (not found in any folder)


...

[ WARN:0@61765.468] global loadsave.cpp:268 findDecoder imread_('/Users/rfalcao/Documents/FYPCusp Images_081224/annotations 4/non_cusp_images/cusp_42.png'): can't open/read file: check file path/integrity
[ WARN:0@61765.469] global loadsave.cpp:268 findDecoder imread_('/Users/rfalcao/Documents/FYPCusp Images_081224/annotations 4/non_cusp_images/cusp_43.png'): can't open/read file: check file path/integrity
[ WARN:0@61765.469] global loadsave.cpp:268 findDecoder imread_('/Users/rfalcao/Documents/FYPCusp Images_081224/annotations 4/non_cusp_images/cusp_44.png'): can't open/read file: check file path/integrity
[ WARN:0@61765.469] global loadsave.cpp:268 findDecoder imread_('/Users/rfalcao/Documents/FYPCusp Images_081224/annotations 4/non_cusp_images/cusp_45.png'): can't open/read file: check file path/integrity
[ WARN:0@61765.469] global loadsave.cpp:268 findDecoder imread_('/Users/rfalcao/Documents/FYPCusp Images_081224/annotations 4/non_cusp_images/cusp_46.png'): can't open/read file: c

Skipping cusp_5.png (not found in any folder)
Skipping cusp_50.png (not found in any folder)
Skipping cusp_51.png (not found in any folder)
Skipping cusp_52.png (not found in any folder)
Skipping cusp_53.png (not found in any folder)
Skipping cusp_54.png (not found in any folder)
Skipping cusp_55.png (not found in any folder)
Skipping cusp_56.png (not found in any folder)
Skipping cusp_57.png (not found in any folder)
Skipping cusp_58.png (not found in any folder)
Skipping cusp_59.png (not found in any folder)
Skipping cusp_6.png (not found in any folder)
Skipping cusp_60.png (not found in any folder)
Skipping cusp_61.png (not found in any folder)
Skipping cusp_62.png (not found in any folder)
Skipping cusp_63.png (not found in any folder)
Skipping cusp_64.png (not found in any folder)
Skipping cusp_65.png (not found in any folder)
Skipping cusp_66.png (not found in any folder)
Skipping cusp_67.png (not found in any folder)
Skipping cusp_68.png (not found in any folder)
Skipping cusp_6

[ WARN:0@61766.084] global loadsave.cpp:268 findDecoder imread_('/Users/rfalcao/Documents/FYPCusp Images_081224/annotations 4/non_cusp_images/cusp_5.png'): can't open/read file: check file path/integrity
[ WARN:0@61766.084] global loadsave.cpp:268 findDecoder imread_('/Users/rfalcao/Documents/FYPCusp Images_081224/annotations 4/non_cusp_images/cusp_50.png'): can't open/read file: check file path/integrity
[ WARN:0@61766.084] global loadsave.cpp:268 findDecoder imread_('/Users/rfalcao/Documents/FYPCusp Images_081224/annotations 4/non_cusp_images/cusp_51.png'): can't open/read file: check file path/integrity
[ WARN:0@61766.084] global loadsave.cpp:268 findDecoder imread_('/Users/rfalcao/Documents/FYPCusp Images_081224/annotations 4/non_cusp_images/cusp_52.png'): can't open/read file: check file path/integrity
[ WARN:0@61766.084] global loadsave.cpp:268 findDecoder imread_('/Users/rfalcao/Documents/FYPCusp Images_081224/annotations 4/non_cusp_images/cusp_53.png'): can't open/read file: ch

Skipping cusp_79.png (not found in any folder)
Skipping cusp_8.png (not found in any folder)
Skipping cusp_80.png (not found in any folder)
Skipping cusp_82.png (not found in any folder)
Skipping cusp_9.png (not found in any folder)
✅ Masks extracted and saved as PNGs!


[ WARN:0@61767.149] global loadsave.cpp:268 findDecoder imread_('/Users/rfalcao/Documents/FYPCusp Images_081224/annotations 4/non_cusp_images/cusp_79.png'): can't open/read file: check file path/integrity
[ WARN:0@61767.150] global loadsave.cpp:268 findDecoder imread_('/Users/rfalcao/Documents/FYPCusp Images_081224/annotations 4/non_cusp_images/cusp_8.png'): can't open/read file: check file path/integrity
[ WARN:0@61767.150] global loadsave.cpp:268 findDecoder imread_('/Users/rfalcao/Documents/FYPCusp Images_081224/annotations 4/non_cusp_images/cusp_80.png'): can't open/read file: check file path/integrity
[ WARN:0@61767.156] global loadsave.cpp:268 findDecoder imread_('/Users/rfalcao/Documents/FYPCusp Images_081224/annotations 4/non_cusp_images/cusp_82.png'): can't open/read file: check file path/integrity
[ WARN:0@61767.177] global loadsave.cpp:268 findDecoder imread_('/Users/rfalcao/Documents/FYPCusp Images_081224/annotations 4/non_cusp_images/cusp_9.png'): can't open/read file: che

In [ ]:
import os
import cv2
import numpy as np

# Paths

padded_image_dir = "/Users/rfalcao/Documents/FYP/ManualSegmentationAnns/padded/images"
padded_mask_dir = "/Users/rfalcao/Documents/FYP/ManualSegmentationAnns/padded/masks"

image_dir = "/Users/rfalcao/Documents/FYP/ManualSegmentationAnns/cusp_images"
mask_dir = "/Users/rfalcao/Documents/FYP/ManualSegmentationAnns/masks"

os.makedirs(padded_image_dir, exist_ok=True)
os.makedirs(padded_mask_dir, exist_ok=True)

# Step 1: Find max width & height in dataset
max_width, max_height = 0, 0
for filename in os.listdir(image_dir):
    img_path = os.path.join(image_dir, filename)
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is not None:
        h, w = img.shape
        max_width = max(max_width, w)
        max_height = max(max_height, h)

print(f"Max dimensions for padding: {max_width}x{max_height}")

# Step 2: Pad each image & mask
for filename in os.listdir(image_dir):
    img_path = os.path.join(image_dir, filename)
    mask_path = os.path.join(mask_dir, filename)

    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    if img is None or mask is None:
        print(f"Skipping {filename} (missing image or mask)")
        continue

    h, w = img.shape
    pad_top = (max_height - h) // 2
    pad_bottom = max_height - h - pad_top
    pad_left = (max_width - w) // 2
    pad_right = max_width - w - pad_left

    # Apply same padding to image and mask
    padded_img = cv2.copyMakeBorder(img, pad_top, pad_bottom, pad_left, pad_right, cv2.BORDER_CONSTANT, value=0)
    padded_mask = cv2.copyMakeBorder(mask, pad_top, pad_bottom, pad_left, pad_right, cv2.BORDER_CONSTANT, value=0)

    # Save results
    cv2.imwrite(os.path.join(padded_image_dir, filename), padded_img)
    cv2.imwrite(os.path.join(padded_mask_dir, filename), padded_mask)

print("✅ Padding complete! All cusp images and masks are now the same size.")
